__Andrew Kittredge__ <br>
__GTECH 731__ <br>
__Assignment 2__

__Task 1:__ Read in a data file of all counties in the US. 

In [1]:
# Read geojson directly from online source
import requests, json
url = "https://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_050_00_5m.json"

with requests.get(url) as response:
    data = response.json()   

__Task 2:__ Find the three most common names of the counties.
- Display the nation-wide frequency of the three most common county names
- List the county names and state codes

In [2]:
# Create a list of features, each of which contains a dictionary of type, properties, and geometry
# Properties and geometry contain their own dictionaries. 
featureList = data['features']

# Create an empty dictionary to store county name and frequency
countyDict = {}

# Loop through each element in featureList, checking if the county name is in the dictionary.
# If present, increment frequency by one
# If not, add to dictionary with frequency = 1

for f in featureList:
    countyName = f['properties']['NAME'] 
    if countyDict.__contains__(countyName):
        curVal = countyDict.get(countyName)
        countyDict.update({countyName: curVal +1})
    else:
        countyDict.update({countyName: 1})

# To get the highest value {key: value} pair in tuple with max() and zip()
# Print frequency and county name
# Then, populate list with state FIPS from counties with matching name. 
# Print list
# Then, remove most frequent key: value pair from dictionary and repeat two more times

for i in range(3):
    maxPair = max(zip(countyDict.values(), countyDict.keys())) # Get max value, key tuple 
    codeList = [None] * maxPair[0] # Create empty list with county frequency
    codeList_idx = 0 # Iterator for index
    print('There are {} {} counties in the US. They are found in states with the following codes:'.format(maxPair[0], maxPair[1]))
    for f in featureList:
        if f['properties']['NAME'] == maxPair[1]:
            codeList[codeList_idx] = (f['properties']['STATE']) # Add FIPS to list
            codeList_idx += 1
        else:
            pass
    print(codeList)
    countyDict.pop(maxPair[1])# Remove highest frequency key, value pair
    

There are 31 Washington counties in the US. They are found in states with the following codes:
['08', '12', '05', '17', '19', '23', '24', '29', '47', '40', '41', '55', '49', '50', '20', '18', '21', '27', '13', '01', '22', '16', '36', '37', '31', '28', '42', '39', '44', '51', '48']
There are 26 Jefferson counties in the US. They are found in states with the following codes:
['17', '13', '19', '41', '08', '05', '12', '20', '21', '01', '22', '28', '16', '18', '29', '47', '48', '36', '40', '31', '30', '42', '39', '54', '55', '53']
There are 26 Franklin counties in the US. They are found in states with the following codes:
['13', '01', '18', '16', '21', '19', '28', '36', '48', '05', '12', '22', '23', '25', '17', '20', '29', '47', '31', '37', '42', '39', '51', '51', '53', '50']


__Task 3:__ Basic statistics by state
For each state, find:
- The number of counties
- The name and size (census area) of the biggest and smallest county by area
- The total and average area of counties

In [3]:
# Get a list of state codes. They are not consecutive and not (1, 51)
stateCodeList = []
for f in featureList:
    stateCode = f['properties']['STATE']
    if stateCodeList.__contains__(stateCode):
        pass
    else:
        stateCodeList.append(stateCode)

# For each state code, get summary statistics
for s in stateCodeList:
    stateCounties = 0 # Initialize county list
    largestCounty = ['initialName', 0] # Initialize for largest county
    areaCounty_total = 0 # Initialize total county area
    areaCounty_avg = 0 # Initialize average county area
    for f in featureList: # Initialize smallest county variable using the first county of each state 
        if f['properties']['STATE'] == s:
            smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            break
        else:
            pass
    for f in featureList: 
        if f['properties']['STATE'] == s: # Filter to currently iterated state code
            stateCounties += 1 # County iterator
            areaCounty_total += f['properties']['CENSUSAREA'] # Add county area to total
            if f['properties']['CENSUSAREA'] > largestCounty[1]: # Largest county comparison
                largestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            elif f['properties']['CENSUSAREA'] < smallestCounty[1]: # Smallest county comparison
                smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            else:
                pass
        else:
            pass
    areaCounty_avg = areaCounty_total / stateCounties # Average county area
        
    print('State {} has {} counties.'.format(s, stateCounties))
    print('Its largest county is {} which is {:.0f} square miles.'.format(largestCounty[0], largestCounty[1]))
    print('Its smallest county is {} which is {:.0f} square miles.'.format(smallestCounty[0], smallestCounty[1]))
    print('Its total county area is {:.0f} square miles.'.format(areaCounty_total))
    print('Its average county area is {:.0f} square miles.\n'.format(areaCounty_avg))
    


State 02 has 29 counties.
Its largest county is Yukon-Koyukuk which is 145505 square miles.
Its smallest county is Skagway which is 452 square miles.
Its total county area is 570641 square miles.
Its average county area is 19677 square miles.

State 04 has 15 counties.
Its largest county is Coconino which is 18619 square miles.
Its smallest county is Santa Cruz which is 1237 square miles.
Its total county area is 113594 square miles.
Its average county area is 7573 square miles.

State 05 has 75 counties.
Its largest county is Union which is 1039 square miles.
Its smallest county is Lafayette which is 528 square miles.
Its total county area is 52035 square miles.
Its average county area is 694 square miles.

State 08 has 64 counties.
Its largest county is Las Animas which is 4773 square miles.
Its smallest county is Broomfield which is 33 square miles.
Its total county area is 103642 square miles.
Its average county area is 1619 square miles.

State 09 has 8 counties.
Its largest count

__Bonus Point__: Using the JSON files for state fips and abbreviations, add state names and abbreviations to the county data above. You are also encouraged to write a new JSON file for the counties with full state name, abbreviation, and FIPS code.

In [4]:
# Load JSON files
with open("stateCodeToFips.json", 'r') as a, open("fipsToState.json", 'r') as b:
    codeToFips = json.load(a)
    fipsToState = json.load(b)

# Invert code to fips dictionary
fipsToCode = {v: k for k, v in codeToFips.items()}

# Modify above code to include state names and abbreviations to county data
for s in stateCodeList:
    stateCounties = 0 # Initialize county list
    largestCounty = ['initialName', 0] # Initialize for largest county
    areaCounty_total = 0 # Initialize total county area
    areaCounty_avg = 0 # Initialize average county area
    stateName = fipsToState.get(s) # Get state name from FIPS to state dictionary
    stateAbbrev = fipsToCode.get(s) # Get state abbreviation from FIPS to code dictionary
    for f in featureList: # Initialize smallest county variable using the first county of each state 
        if f['properties']['STATE'] == s:
            smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            break # This populates the list with only one county, area pair
        else:
            pass
    for f in featureList: 
        if f['properties']['STATE'] == s: # Filter to currently iterated state code
            stateCounties += 1 # County iterator
            areaCounty_total += f['properties']['CENSUSAREA'] # Add county area to total
            if f['properties']['CENSUSAREA'] > largestCounty[1]: # Largest county comparison
                largestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            elif f['properties']['CENSUSAREA'] < smallestCounty[1]: # Smallest county comparison
                smallestCounty = [f['properties']['NAME'], f['properties']['CENSUSAREA']]
            else:
                pass
        else:
            pass
    areaCounty_avg = areaCounty_total / stateCounties # Average county area
        
    print('{} ({}) has {} counties.'.format(stateName, stateAbbrev, stateCounties))
    print('Its largest county is {} which is {:.0f} square miles.'.format(largestCounty[0], largestCounty[1]))
    print('Its smallest county is {} which is {:.0f} square miles.'.format(smallestCounty[0], smallestCounty[1]))
    print('Its total county area is {:.0f} square miles.'.format(areaCounty_total))
    print('Its average county area is {:.0f} square miles.\n'.format(areaCounty_avg))

Alaska (AK) has 29 counties.
Its largest county is Yukon-Koyukuk which is 145505 square miles.
Its smallest county is Skagway which is 452 square miles.
Its total county area is 570641 square miles.
Its average county area is 19677 square miles.

Arizona (AZ) has 15 counties.
Its largest county is Coconino which is 18619 square miles.
Its smallest county is Santa Cruz which is 1237 square miles.
Its total county area is 113594 square miles.
Its average county area is 7573 square miles.

Arkansas (AR) has 75 counties.
Its largest county is Union which is 1039 square miles.
Its smallest county is Lafayette which is 528 square miles.
Its total county area is 52035 square miles.
Its average county area is 694 square miles.

Colorado (CO) has 64 counties.
Its largest county is Las Animas which is 4773 square miles.
Its smallest county is Broomfield which is 33 square miles.
Its total county area is 103642 square miles.
Its average county area is 1619 square miles.

Connecticut (CT) has 8 co

In [20]:
# Create a copy of feature list to update and overwrite in copy of original dictionary
featureList_Update = featureList

# Use a for loop to update counties with state name and abbreviation
update_idx = 0 #iterator
for f in featureList_Update:
    stateName = fipsToState.get(f['properties']['STATE'])
    stateAbbrev = fipsToCode.get(f['properties']['STATE'])
    featureList_Update[update_idx]['properties'].update({'CODE': stateAbbrev, 'STATENAME': stateName})
    update_idx += 1

# Create copy of originial dictionary    
dataUpdate = data
dataUpdate['features'] = featureList_Update # Overwrite updated features

# Write new json file
with open("countiesUpdate.json", "w") as outfile:
    json.dump(dataUpdate, outfile)